# ABS Quarterly Living Cost Index 6467

## Python set-up

In [1]:
# system imports
import textwrap

# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol

# local imports
from abs_helper import get_abs_data, ANNUAL_CPI_TARGET_RANGE, QUARTERLY_CPI_TARGET
from mgplot import line_plot_finalise, growth_plot_finalise, multi_start

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

NOTE: A bit ugly - the ABS does not appear to provide a complete summary zip-file for this one.

In [2]:
abs_dict, meta, source, _ = get_abs_data("6467.0", get_zip=False, get_excel=True)


In [3]:
textwrap.wrap(", ".join(abs_dict.keys()), 80)

['646701, 646702, 646703']

## Plot

### Headline LCI for Australia

In [4]:
QUARTERLY_PHRASE = "Percentage Change from Previous Period"
ANNUAL_PHRASE = "Percentage Change from Corresponding Quarter of Previous Year"


def get_labels(table="646701"):

    return [
        x[1].strip()
        for x in meta[
            meta[metacol.did].str.contains(ANNUAL_PHRASE)
            & (meta[metacol.table] == table)
        ][metacol.did].str.split(";")
    ]

In [5]:
def plot_lci() -> None:

    table = "646701"
    data = abs_dict[table]
    print(data.index[0], data.index[-1])

    # Used for pattern matching retrieval

    labels = get_labels(table)

    annuals = {}
    for label in labels:
        core_search = {
            table: metacol.table,
            label: metacol.did,
        }
        _, a_id, _ = ra.find_abs_id(meta, core_search | {ANNUAL_PHRASE: metacol.did})
        _, q_id, _ = ra.find_abs_id(meta, core_search | {QUARTERLY_PHRASE: metacol.did})
        annuals[label] = data[a_id]  # for the second plot below
        growth = pd.DataFrame({"Annual": data[a_id], "Quarterly": data[q_id]})
        multi_start(
            growth,
            starts=[0, -15],
            function=growth_plot_finalise,
            title=textwrap.fill(f"Living Cost Index: {label}", 60),
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            axhline=QUARTERLY_CPI_TARGET,
            legend={"fontsize": 8, "loc": "best"},
            rfooter=source,
            lfooter=f"Australia. ",
            show=SHOW,
        )

    line_plot_finalise(
        pd.DataFrame(annuals),
        title=f"Annual Growth in Living Cost Indexes",
        ylabel="Per cent",
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        legend={"fontsize": 8, "loc": "best", "ncol": 2},
        rfooter=f"{source} {table}",
        lfooter="Australia. ",
        show=SHOW,
    )


plot_lci()

1998Q2 2025Q1


### LCI components

In [6]:
def components() -> None:

    table = "646703"
    data = abs_dict[table]

    typelabels = set(
        [
            x[2].strip()
            for x in meta[
                meta[metacol.did].str.contains(ANNUAL_PHRASE)
                & (meta[metacol.table] == table)
            ][metacol.did].str.split(";")
        ]
    )

    for typelabel in typelabels:
        annuals = {}
        for label in get_labels():
            _, a_id, _ = ra.find_abs_id(
                meta,
                {
                    table: metacol.table,
                    ANNUAL_PHRASE: metacol.did,
                    label: metacol.did,
                    typelabel: metacol.did,
                },
                # verbose=True,
            )
            annuals[label] = data[a_id]

        line_plot_finalise(
            pd.DataFrame(annuals),
            title=f"LCI Annual Growth: {typelabel}",
            ylabel="Per cent",
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            legend={"fontsize": "xx-small", "ncol": 2},
            rfooter=f"{source} {table}",
            lfooter="Australia. ",
            show=SHOW,
        )


components()

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-06-06 18:06:10

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.3.0

conda environment: n/a

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas : 2.2.3
mgplot : 0.1.7a3
readabs: 0.0.31

Watermark: 2.5.0



In [8]:
print("Finished")

Finished
